# 📚 Populando o Pinecone com Base de Conhecimento

**Objetivo**: Indexar informações turísticas de Rio de Janeiro e Paris para uso no sistema RAG.

## 📋 Ordem de Execução:
1. **Célula 2**: Conectar ao Pinecone
2. **Célula 3**: Carregar e dividir texto em chunks
3. **Célula 4**: Carregar modelo de embeddings (demora na 1ª vez)
4. **Célula 5**: Gerar embeddings e inserir no Pinecone
5. **Célula 6**: Testar busca por similaridade

## ✅ Status Atual:
- ✅ Índice 'guia-viagem' criado (384 dimensões)
- ✅ 4 chunks indexados com metadados
- ✅ Busca por similaridade funcionando

In [1]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Conectar ao Pinecone
pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
indice = pinecone_client.Index('guia-viagem')

print("Conectado ao índice 'guia-viagem' com sucesso!")

Conectado ao índice 'guia-viagem' com sucesso!


In [5]:
# Carregar base de conhecimento e dividir em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np

# Carregar base de conhecimento
with open('base_conhecimento_rio_paris.txt', 'r', encoding='utf-8') as f:
    texto = f.read()

# Separar em chunks (ajuste chunk_size conforme necessário)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.create_documents([texto])

print(f"✅ Texto dividido em {len(chunks)} chunks.")
print(f"📄 Exemplo do primeiro chunk: {chunks[0].page_content[:100]}...")
print(f"📏 Tamanho médio dos chunks: {sum(len(chunk.page_content) for chunk in chunks) // len(chunks)} caracteres")

✅ Texto dividido em 4 chunks.
📄 Exemplo do primeiro chunk: # Base de Conhecimento - Rio de Janeiro
Pão de Açúcar: Um dos pontos turísticos mais famosos do Rio,...
📏 Tamanho médio dos chunks: 211 caracteres


In [6]:
# Carregar modelo de embeddings (pode demorar na primeira vez)
from langchain_huggingface import HuggingFaceEmbeddings

print("🔄 Carregando modelo de embeddings...")
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
print("✅ Modelo carregado com sucesso!")

🔄 Carregando modelo de embeddings...
✅ Modelo carregado com sucesso!
✅ Modelo carregado com sucesso!


In [7]:
# Gerar embeddings e inserir no Pinecone
print("🔄 Gerando embeddings dos chunks...")

# Preparar dados para inserção
vetores_para_inserir = []

for i, chunk in enumerate(chunks):
    print(f"📊 Processando chunk {i+1}/{len(chunks)}", end="\r")
    
    # Gerar embedding
    embedding = embeddings_model.embed_documents([chunk.page_content])[0]
    
    # Preparar vetor com metadados
    vetor = {
        'id': f'chunk_{i}',
        'values': embedding,
        'metadata': {
            'text': chunk.page_content,
            'source': 'base_conhecimento_rio_paris.txt',
            'chunk_index': i
        }
    }
    vetores_para_inserir.append(vetor)

print(f"\n✅ Embeddings gerados para {len(chunks)} chunks!")

# Inserir no Pinecone
print("🔄 Inserindo vetores no Pinecone...")
indice.upsert(vectors=vetores_para_inserir)
print("✅ Vetores inseridos com sucesso!")

# Verificar estatísticas
stats = indice.describe_index_stats()
print(f"📊 Estatísticas do índice: {stats}")

🔄 Gerando embeddings dos chunks...
📊 Processando chunk 4/4
✅ Embeddings gerados para 4 chunks!
🔄 Inserindo vetores no Pinecone...

✅ Embeddings gerados para 4 chunks!
🔄 Inserindo vetores no Pinecone...
✅ Vetores inseridos com sucesso!
📊 Estatísticas do índice: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4,
 'vector_type': 'dense'}
✅ Vetores inseridos com sucesso!
📊 Estatísticas do índice: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4,
 'vector_type': 'dense'}


In [8]:
# Testar busca por similaridade
query = "Quais são os pontos turísticos do Rio de Janeiro?"
query_embedding = embeddings_model.embed_documents([query])[0]

# Buscar documentos similares
resultado = indice.query(vector=query_embedding, top_k=3, include_metadata=True)

print("🔍 Testando busca por similaridade:")
print(f"Consulta: {query}")
print("\nResultados encontrados:")
for match in resultado['matches']:
    print(f"ID: {match['id']}")
    print(f"Similaridade: {match['score']:.3f}")
    print(f"Texto: {match['metadata']['text'][:150]}...")
    print("-" * 50)

🔍 Testando busca por similaridade:
Consulta: Quais são os pontos turísticos do Rio de Janeiro?

Resultados encontrados:
ID: chunk_0
Similaridade: 0.729
Texto: # Base de Conhecimento - Rio de Janeiro
Pão de Açúcar: Um dos pontos turísticos mais famosos do Rio, com vista panorâmica da cidade.
Cristo Redentor: ...
--------------------------------------------------
ID: chunk_1
Similaridade: 0.505
Texto: Restaurante Aprazível: Restaurante de comida brasileira contemporânea em Santa Teresa.
Dica: Evite ostentar objetos de valor em áreas turísticas movim...
--------------------------------------------------
ID: chunk_2
Similaridade: 0.431
Texto: # Base de Conhecimento - Paris
Torre Eiffel: Principal símbolo de Paris, ideal para visitação diurna e noturna.
Museu do Louvre: Maior museu de arte d...
--------------------------------------------------
